In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='52KYU1B4HW8FCKX0')

In [2]:
class ScriptData:

    def __init__(self, fetched_data = {}, converted_data = {}):
        self.fetched_data = fetched_data
        self.converted_data = converted_data

    # script : accepts script_name for converted_data and (script_name, 'fetched') for fetched_data
    # returns fetched raw data or converted DataFrame for script
    def __getitem__(self, script):
        try:
            script, data_type = script
            try:
                return self.fetched_data[script]
            except KeyError:
                return
        except ValueError:
            try:
                return self.converted_data[script]
            except KeyError:
                return 'converted intraday data not available'

    # saves data into the appropriate dictionary as per the type of data
    def __setitem__(self, script, data):
        if type(data) is pd.DataFrame:
            self.converted_data[script] = data
        else:
            self.fetched_data[script] = data

    # checks if data fo script exists in converted_data
    def __contains__(self, script):
        if script in self.converted_data:
            return True
        return False

    # fetches raw data for a script
    def fetch_intraday_data(self, script):
        data, metadata = ts.get_intraday(symbol=script)
        self[script] = data

    # converts raw data into a organised pandas DataFrame with required columns and datatypes
    def convert_intraday_data(self, script):
        data = self[script, 'fetched']
        if data is None:
            return 'fetch data for this stock first'

        # Arranging dataframe
        df = pd.DataFrame(data).T
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df.insert(0, 'timestamp', df.index)
        df.index = np.arange(len(data))

        # Adopting appropriate datatypes
        df = df.astype({'open': 'float', 'high': 'float', 'low': 'float', 'close': 'float', 'volume': 'int'})
        df['timestamp'] = pd.to_datetime(df.timestamp)

        self[script] = df

In [3]:
script_data = ScriptData()

In [4]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-22 20:00:00,91.9000,92.0300,91.8100,92.0300,9548
1,2023-02-22 19:45:00,91.9500,91.9600,91.8300,91.8400,12187
2,2023-02-22 19:30:00,91.8900,91.9500,91.8500,91.9400,3537
3,2023-02-22 19:15:00,91.8400,91.9300,91.8100,91.8900,12061
4,2023-02-22 19:00:00,91.8200,91.8400,91.7500,91.8400,2516
...,...,...,...,...,...,...
95,2023-02-21 12:15:00,92.1500,92.3300,92.0400,92.0900,816760
96,2023-02-21 12:00:00,92.3800,92.4000,92.1200,92.1550,575991
97,2023-02-21 11:45:00,92.2600,92.3900,92.0750,92.3700,762560
98,2023-02-21 11:30:00,92.3268,92.3368,92.1600,92.2500,557566


In [5]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-22 20:00:00,149.130,149.240,149.10,149.1800,8726
1,2023-02-22 19:45:00,149.230,149.250,149.11,149.1500,13304
2,2023-02-22 19:30:00,149.270,149.290,149.18,149.1900,13253
3,2023-02-22 19:15:00,149.180,149.300,149.18,149.3000,1546
4,2023-02-22 19:00:00,149.180,149.240,149.06,149.1600,6135
...,...,...,...,...,...,...
95,2023-02-21 12:15:00,149.800,149.990,149.65,149.7600,1265926
96,2023-02-21 12:00:00,149.800,149.940,149.52,149.8001,1139047
97,2023-02-21 11:45:00,149.505,149.840,149.36,149.8050,1201495
98,2023-02-21 11:30:00,149.785,149.845,149.42,149.5100,1352613


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
# function to calculate indicator : average for previous specified closing data
# returns a pandas DataFrame having indicator along with timestamps
def indicator1(df, timeperiod):
    new_df = df.loc[:,['timestamp']]
    close_series = df.loc[:, 'close']

    # Using rolling function to calculate mean of previous 'timeperiod' number of rows in close column
    rolling_average = close_series.rolling(window=timeperiod, min_periods=timeperiod).mean()

    new_df.insert(1, 'indicator', rolling_average)
    return new_df

In [10]:
indicator1(script_data['GOOGL'], 5)

,timestamp,indicator
0,2023-02-22 20:00:00,NaN
1,2023-02-22 19:45:00,NaN
2,2023-02-22 19:30:00,NaN
3,2023-02-22 19:15:00,NaN
4,2023-02-22 19:00:00,91.90800
...,...,...
95,2023-02-21 12:15:00,91.91986
96,2023-02-21 12:00:00,91.99712
97,2023-02-21 11:45:00,92.08312
98,2023-02-21 11:30:00,92.15912


In [11]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
0,2023-02-22 20:00:00,NaN
1,2023-02-22 19:45:00,NaN
2,2023-02-22 19:30:00,NaN
3,2023-02-22 19:15:00,NaN
4,2023-02-22 19:00:00,149.19600
...,...,...
95,2023-02-21 12:15:00,149.82050
96,2023-02-21 12:00:00,149.84252
97,2023-02-21 11:45:00,149.79952
98,2023-02-21 11:30:00,149.71152


In [12]:
class Strategy:

    def __init__(self, script, intraday_data={}):
        self.script = script
        self.intraday_data = intraday_data
    
    # fetches data from class ScriptData and stores it
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.intraday_data = script_data[self.script]

    # returns a pandas DataFrame providing with BUY/SELL signal comparing indicator data and closing data of scripts
    def get_signals(self):
        indicator_data = indicator1(self.intraday_data, 5)
        indicator_data['close_data'] = self.intraday_data.loc[:, 'close']
    
        # BUY when following indicator_data is greater than following close_data and previous indicator_data is less than previous close_data
        buy_condition = (indicator_data['indicator'].shift() > indicator_data['close_data'].shift()) & (indicator_data['indicator'].shift(-1) < indicator_data['close_data'].shift(-1))

        # SELL when previous indicator_data is greater than previous close_data and following indicator_data is less than following close_data
        sell_condition = (indicator_data['indicator'].shift(-1) > indicator_data['close_data'].shift(-1)) & (indicator_data['indicator'].shift() < indicator_data['close_data'].shift())

        # NO SIGNAL otherwise
        indicator_data['signal'] = np.select([buy_condition, sell_condition, True], ['BUY', 'SELL', 'NO SIGNAL'])

        # New dataframe 'signals' with columns 'timestamp' and 'signal' tracking only BUY/SELL rows 
        signals = indicator_data.loc[:, ['timestamp', 'signal']]
        signals.drop(signals[signals.signal=='NO SIGNAL'].index, inplace = True)
        signals.reset_index(drop=True, inplace = True)
    
        return signals

In [13]:
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2023-02-22 18:30:00,SELL
1,2023-02-22 17:30:00,BUY
2,2023-02-22 17:00:00,SELL
3,2023-02-22 15:00:00,BUY
4,2023-02-22 14:45:00,BUY
5,2023-02-22 13:30:00,SELL
6,2023-02-22 13:15:00,SELL
7,2023-02-22 10:30:00,BUY
8,2023-02-22 10:15:00,BUY
9,2023-02-22 09:45:00,SELL


In [14]:
from pyalgotrading.utils.func import plot_candlestick_chart
from pyalgotrading.constants import PlotType

In [15]:
# function to plot a candlestick graph between script data(open, high, low, close, volume) and indicator data for a given script and timeperiod of indicator
# uses plot_candlestick_chart function of pyalgotrading library for the same
def plot_candlestick_graph(script, timeperiod):
    df = script_data[script].sort_values(by='timestamp')
    indicator_data = indicator1(script_data[script], 5).sort_values(by='timestamp')
    plot_candlestick_chart(df, PlotType.HEIKINASHI, hide_missing_dates=True, show=True, indicators=[{'name': 'SMA', 'data': indicator_data['indicator']}])

In [16]:
plot_candlestick_graph('GOOGL', 5)